# A very simple power system model tutorial

This notebook provides a very simple tutorial into some of the functionality found in the models in this repository. The models are simple: they meet demand with four generation technologies (baseload (cf. nuclear), peaking (cf. gas), wind, and solar), as well as the possibility of energy storage.

This notebook is only intended as a very quick introduction. For more extensive documentation, including the optimisation problem that these models solve, see the `documentation` directory.

In [1]:
# Suppress minor warnings
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

# Import the models
import psm

# One region model

We start with a model with just one region, viewing the whole system as a "copper plate".

### Import time series data

In [2]:
ts_data = psm.utils.load_time_series_data('1_region')
ts_data = ts_data.loc['2017-06-08':'2017-06-14']
ts_data.head(10)

,demand,wind,solar
t,,,
2017-06-08 00:00:00,28.38,0.3456,0.0000
2017-06-08 01:00:00,27.04,0.3624,0.0000
2017-06-08 02:00:00,26.13,0.3609,0.0000
2017-06-08 03:00:00,25.61,0.3572,0.0001
2017-06-08 04:00:00,26.06,0.3480,0.0124
2017-06-08 05:00:00,30.16,0.3624,0.0426
2017-06-08 06:00:00,35.57,0.4005,0.0790
2017-06-08 07:00:00,39.23,0.4608,0.1340
2017-06-08 08:00:00,41.01,0.5247,0.1919


### `Operate` mode: meet demand with a fixed system

In this example, the generation capacities are user-defined, and the model optimises how demand is met. This is sometimes called the *economic dispatch* or the *unit commitment* problem.

In [5]:
# Set installed capacities
fixed_caps = {
    'cap_baseload_total': 0.,  # MW
    'cap_peaking_total': 0.,  # MW 
    'cap_wind_total': 0.,  # MW 
    'cap_solar_total': 0.,  # MW
    'cap_storage_power_total': 0.,  # MW
    'cap_storage_energy_total': 0.1  # MWh
}

# Create the model with the fixed capacities
model = psm.models.OneRegionModel(
    ts_data=ts_data,
    run_mode='operate',
    fixed_caps=fixed_caps,
    allow_unmet=True
)

In [6]:
# Run the model, solve the optimisation problem
model.run()

In [ ]:
# Show a summary of outputs
model.get_summary_outputs()

In [ ]:
# Show key time series outputs
model.get_timeseries_outputs()

In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots are sensitive to jupyter notebook settings, and may not display
# For more robust plots, use the HTML webpage created in `scripts/main.py`
model.plot.timeseries(array='all', subset={'costs': ['monetary']})

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_operate')
model.to_csv('outputs_operate')

## `Plan` mode: design cost-optimal system

In this example, the model optimises both the design and operation of a system. This is sometimes called the *capacity expansion planning* problem.

In [ ]:
# Create the model without any fixed capacities
model = psm.models.OneRegionModel(ts_data=ts_data, run_mode='plan')

In [ ]:
# Run the model, solve the optimisation problem
model.run()

In [ ]:
# Show a summary of outputs
model.get_summary_outputs()

In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots are sensitive to jupyter notebook settings, and may not display
# For more robust plots, use the HTML webpage created in `scripts/main.py`
model.plot.timeseries(array='all', subset={'costs': 'monetary'})

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_plan')
model.to_csv('outputs_plan')

# 6 region model

<img align="right" src="documentation/6_region_diagram.jpg" alt="drawing" width="300" height="250">

We can do the same with a model with 6 regions. In this system, supply and demand must match across the model as a whole but electricity may be transmitted around the grid according to a topology inspired by the *IEEE 6-bus test system* and [Kamalinia & Shahidehpour (2010)](https://doi.org/10.1049/iet-gtd.2009.0695). The regions contain the following demand and generation technologies:
- Region 1: baseload & peaking generation
- Region 2: demand, wind and solar generation, with time series from Germany
- Region 3: baseload & peaking generation
- Region 4: demand, with time series from France
- Region 5: demand, wind and solar generation, with time series from the United Kingdom
- Region 6: baseload, peaking, wind and solar generation, with time series from Spain

Transmission is permitted between regions 1-2, 1-5, 1-6, 2-3, 3-4, 4-5 and 5-6.

### Load time series data

In [ ]:
ts_data = psm.utils.load_time_series_data('6_region')
ts_data = ts_data.loc['2017-06-07':'2017-06-14']
ts_data[ts_data < 0] = 0
ts_data.head(10)

### Run the model in `plan` mode

In [ ]:
model = psm.models.SixRegionModel(ts_data=ts_data, run_mode='plan')
model.run()
display(model.get_summary_outputs())
display(model.get_timeseries_outputs())

In [ ]:
# Note: these plots are sensitive to jupyter notebook settings, and may not display
# For more robust plots, use the HTML webpage created in `scripts/main.py`
model.plot.timeseries(array='all', subset={'costs': 'monetary'})

## A note on `operate` mode for the `6_region` model

To run the 6-region model in `operate` mode, you'll have to set all the generation and transmission capacities in the relevant regions. To figure out the dictionary keys, you can run the model in `plan` mode and use `model.get_summary_outputs()`. From there, you have to use all rows with index of the form `cap_*_region*` and `cap_transmission_region*_region*`. For example:

```
fixed_caps = {
    'cap_baseload_region1': 10.,
    'cap_peaking_region1': 5.,
    'cap_transmission_region1_region2': 15.,
    'cap_transmission_region1_region5': 12.,
    'cap_transmission_region1_region6': 10.,
    'cap_wind_region2':	32.,
    ...
}
```